In [ ]:
# if using colab run this
!git clone https://github.com/HAVIGILI/Anomaly_understanding.git
%cd Anomaly_understanding

In [ ]:
%pip install -r requirements.txt

In [ ]:
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
import torch
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_transform = transforms.Compose([
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
validation_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

train_dataset = datasets.CIFAR10(root="./data", train=True, transform=train_transform, download=True)
validation_dataset = datasets.CIFAR10(root="./data", train=False, transform=validation_transform, download=True)
print("length of train data", len(train_dataset), "length of validation data", len(validation_dataset), "nr of classes", len(train_dataset.classes))

cifar_train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
cifar_validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [ ]:
#import torchvision
#from custommodel import CustomModel

#cnn_model = CustomModel(32, len(train_dataset.classes))

In [ ]:
#from trainer import Trainer

#ModelTrainer = Trainer(cnn_model, cifar_train_loader, cifar_validation_loader, 10, 0.001)
#ModelTrainer.train()

In [ ]:
from pytorch_ood.model import WideResNet
from torchvision import datasets
from pytorch_ood.utils import ToUnknown
import random
from torch.utils.data import Subset

trans = WideResNet.transform_for("cifar10-pt")

class ToTensorTarget:
    def __call__(self, y):
        return torch.tensor(y)

import torchvision.transforms as T

target_transform = T.Compose([
    ToUnknown(),
    ToTensorTarget()
])

ood_dataset1 = datasets.SVHN(root="data", split="test", transform=trans, download=True, target_transform=ToUnknown())
ood_dataset2 = datasets.FakeData(size=10000, image_size=(3, 32, 32), num_classes=10, transform=trans, target_transform=target_transform)
ood_dataset3 = datasets.MNIST(root="data", train=False, transform=trans, download=True, target_transform=ToUnknown())
ood_dataset4 = datasets.CIFAR100(root="data", train=False, transform=trans, download=True, target_transform=ToUnknown())
ood_dataset5 = datasets.FashionMNIST(root="data", train=False, transform=trans, download=True, target_transform=ToUnknown())
ood_dataset6 = datasets.STL10(root="data", split="test", transform=trans, download=True, target_transform=ToUnknown())

id_dataset = datasets.CIFAR10(root="./data", train=False, transform=validation_transform, download=True)

ood_datasets = [ood_dataset1, ood_dataset2, ood_dataset3, ood_dataset4, ood_dataset5, ood_dataset6]  # Add all your OOD datasets here

desired_size = 5000  # Set your desired size

for i, ood_dataset in enumerate(ood_datasets):
    all_indices = list(range(len(ood_dataset)))
    random.shuffle(all_indices)
    subset_indices = all_indices[:desired_size]
    subset = Subset(ood_dataset, subset_indices)
    ood_datasets[i] = subset

In [ ]:
from anomaly_detector import AnomalyDetector

model = WideResNet(num_classes=10, pretrained="cifar10-pt").to(device).eval()
calibration_dataset = train_dataset
id_test_dataset = id_dataset

for i, ood_dataset in enumerate(ood_datasets):
    print("OOD dataset", i)
    ood_test_dataset = ood_dataset
    detector = AnomalyDetector(model, calibration_dataset, id_test_dataset, ood_test_dataset, device=device)

    layers = [model.conv1, model.block1, model.block2, model.block3, nn.Sequential(model.bn1, model.relu)]
    head = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Flatten(), model.fc)

    print("multimahalanobis distance results:")
    detector.multimahalanobis(layers, True)

    print("mahalanobis distance results:")
    detector.mahalanobis(True)

    # print("openmax results:")
    # detector.openmax(True)

    # print("gram results:")
    # detector.gram(head, layers, 10, [1, 2, 3, 4, 5], True)

    # print("maxsoftmax results:")
    # detector.maxsoftmax(True)

    # print("mcd results:")
    # detector.mcd(30, True)

In [ ]:
import requests

# URL of the plane image
url = "https://th.bing.com/th/id/R.21621d8860f8aa6040a48c551a930de2?rik=3uoSLAbD6Voriw&riu=http%3a%2f%2fjamsdesignsinc.com%2fwp-content%2fuploads%2f2018%2f06%2fAirplane_01-square-1024x1024.jpg&ehk=n5hvZiqC3bgBZZ3z9tNUuH%2fBdsLAQFf%2bb2atiLN4Vx0%3d&risl=&pid=ImgRaw&r=0"

# Download the image
response = requests.get(url)
if response.status_code == 200:
    with open("plane.jpg", "wb") as f:
        f.write(response.content)
    print("Image downloaded: plane.jpg")
else:
    print("Failed to download image")

from PIL import Image
from torchvision import transforms

# Load the downloaded image
img = Image.open("plane.jpg")

# Define CIFAR-10 preprocessing: resize, convert to tensor, and normalize
preprocess = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to 32x32 pixels
    transforms.ToTensor(),       # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize (CIFAR-10 stats)
])

# Preprocess the image
input_tensor = preprocess(img).unsqueeze(0)  # Add batch dimension
print("Preprocessed image tensor:", input_tensor.shape)

# Load your trained model
cnn_model.eval()  # Ensure the model is in evaluation mode

# Pass the image through the model
with torch.no_grad():
    output = cnn_model(input_tensor.to(device))
    predicted_class = output.argmax(dim=1).item()

# CIFAR-10 class labels
class_labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# Print the prediction
print("Predicted class:", class_labels[predicted_class])
